# Wasps Replicator 11 Notebook

File name: wasps_replicator_11.ipynb.

Edited with minor revisions on 2017-09-01 and subsequently.

Checked and cleaned up a bit by SOK on 2017-09-06.

Always handy: https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet
        
Use the `go()` procedure at the bottom to run the code. The input values
for the model are presented in the procedure definition. Editing the procedure
is an excellent way to alter parameter values for a given run. 

Code high-level restructuring: 
    
**initialize** Initialize parameter values create an instance of object of class parameters 
    that holds them. The instance is called ``params``. **initialize** sets the default
    parameter values as discussed in the paper.
    
**setup** Set up the system, using the parameter values from ``params``, including, e.g., create agents,
    write headers to files, etc.
    
**run** Does one run.
    
**replicate** Does multiple runs.
    
**sweep** Does multiple runs with different parameter settings. By default we do 30 replications 
for each setting of the parameters and each run by default has 2,000,000 steps (ticks). Also in **sweep**
a nested for-loop is used systemacially to sweep through combinations of parameter values.

Use **go** (at the bottom) to undertake multiple runs in batch.

A run is an execution of the model with a fixed parameter setting.

Runs may be and usually are **replicated**

runLengthInTicks is how many ticks long a run is.

numReplications is how many replications are to be done for each run.

replicationCount  is   how many replications have be
completed for the present run.

runCount is  the number of runs completed.

In the output files, it's runCounter-replicationNum

** Be sure to use random.random, etc. instead of np.random. etc.**

## Code Description

Our code is written in Python 3. 

### The ``wasp`` class

The ``wasp`` class is central. A list, called ``wasps`` and consisting of
*N* ``wasp`` class instances (*N* = 3000 in our standard runs), 
is the main data structure for the simulation. Each ``wasp`` instance 
is initialized with an ``aggression`` value and a ``character`` value. 
``aggression`` values may be 0 (low aggression), 1 (medium aggression), 
and 2 (high aggression). In addition, each ``wasp`` instance may have 
character value of 0 (the individual is a liar) or 1 (the individual is 
honest). At initialization in our standard runs there are 1000 each 
of the 3 aggression types and character values are assigned randomly.

Each ``wasp`` instance has a derived attribute called ``signal``, 
indicating which signal it utters when it encounters another ``wasp``
instance. Honest wasps that have aggression value 2 (high aggression) 
signal RED, while honest wasps with aggression 1 signal YELLOW, and 
honest low aggression wasps signal GREEN. Dishonest wasps of high 
aggression are given a ``signal`` value of YELLOW or GREEN with equal 
probability. Dishonest wasps of medium aggression get a ``signal``
value of RED or GREEN with equal probability. Finally, Dishonest 
wasps of low aggression get a signal of RED or YELLOW with equal 
probability.

See class Parameters for the parameter values given to wasps.


### Do the imports and declare globals

In [1]:
import numpy as np
import  random # We want to be consistent about the random number generator.
# For now, random, instead of np.random, but either way is ok.
import os, copy
import datetime

## Initialization

Declare Parameters class, instantiate a Parameters object, 
params, then call initialize() with it.

In [2]:
class Parameters():
    
    def __init__(self):
        '''
        Initialize with a default set of parameters. We will try to keep
        this class method-free for possible later conversion to Julia.
        Perhaps use a dictionary in which the keys are string
        versions of the parameter names. However, we don't take this 
        stricture very seriously. We like Python. Anyway the policy is to
        maintain all of the parameter values here. If they need to be changed
        during a run, fine. Normally do a deepcopy and change the new copy.
        '''
        self.runCount = 0 # This is used in sweep() to identify each parameter setting
        # The number of runs (distinct parameter settings exeuctions, not replications)
        # completed.
        # Should this parameter be stored and held outside of Parameters()? For now, it's in.
        self.numReplications = 10 # The number of replications to be done of each run.
        # Number of ticks, i.e., paired encounters, in a single run:
        self.runLengthInTicks =  1000000 # is the default for real runs. 50000 for testing.
        self.replicationCount = 0 # This is used in replicate() to keep a running tab on
        # the replication count. Counter of replications.
        self.tick = 0 # Each run goes for a number of ticks, set by runLengthInTicks. 
        # We put the outputs in a standard place, with a timestamp for uniqueness:
        self.outputRoot = '../outputs/Run2016-09-12' + str(datetime.datetime.now())
        self.bdlogfile = 'birthanddeathlog.csv'
        self.bdlogging = False
        self.runDiagnosticsFile='rundiagnostics.csv'
        self.recordRunDiagnostics = True
        self.runParametersLog = 'runparameterslog.txt' #formerly runlog.txt
        self.recordRunParameters = True
        
        # These are the default sweep parameters, to be
        # changed as needed for any given sweep.
        self.aggressionResList=[1.0, 0.95, 0.9, 0.85]
        self.metCostFactorsList=[1.0, 0.95, 0.9, 0.85]
        self.resourceHonestAppropriationsList=[1.0, 0.95, 0.9, 0.85]
        self.resourceDisHonestAppropriationsList=[1.0, 0.95, 0.9, 0.85]
        
        # aggressionResources measures the initial endowments of resources
        # (against metabolic costs, etc.) of the wasps, by aggression
        # level, 0 (low aggerssion), 1, and 2 (high aggression).
        self.aggressionResources = {} # dictionary of resources at init for 
        # various levels of aggression.
        self.aggressionResources[0] = 50
        self.aggressionResources[1] = 75
        self.aggressionResources[2] = 100
            
        # Metabolic costs by level of aggression.
        self.metCostFactors = {}
        self.metCostFactors[0] = 0.05
        self.metCostFactors[1] = 0.075
        self.metCostFactors[2] = 0.1
                
        # Proportion of resources taken from defeated honest agents.
        self.resourceHonestAppropriations = {}
        self.resourceHonestAppropriations[0] = 0.25
        self.resourceHonestAppropriations[1] = 0.5
        self.resourceHonestAppropriations[2] = 0.75
        
        # Proportion of resources taken from defeated dishonest agents.
        self.resourceDisHonestAppropriations = {}
        self.resourceDisHonestAppropriations[0] = 0.5
        self.resourceDisHonestAppropriations[1] = 0.75
        self.resourceDisHonestAppropriations[2] = 1.0
        
        # Number of wasps in the total population.
        self.popSize = 3000
        
        self.outputFolder = self.outputRoot + os.sep + 'RunDataMulti'
        self.recordEvery = 1000
        

In [3]:
class wasp():
    '''
    The ``wasp`` class is central. A list, called ``wasps`` and consisting of
*N* ``wasp`` class instances (*N* = 3000 in our standard runs), 
is the main data structure for the simulation. Each ``wasp`` instance 
is initialized with an ``aggression`` value and a ``character`` value. 
``aggression`` values may be 0 (low aggression), 1 (medium aggression), 
and 2 (high aggression). In addition, each ``wasp`` instance may have 
character value of 0 (the individual is a liar) or 1 (the individual is 
honest). At initialization in our standard runs there are 1000 each 
of the 3 aggression types and character values are assigned randomly.

Each ``wasp`` instance has a derived attribute called ``signal``, 
indicating which signal it utters when it encounters another ``wasp``
instance. Honest wasps that have aggression value 2 (high aggression) 
signal RED, while honest wasps with aggression 1 signal YELLOW, and 
honest low aggression wasps signal GREEN. Dishonest wasps of high 
aggression are given a ``signal`` value of YELLOW or GREEN with equal 
probability. Dishonest wasps of medium aggression get a ``signal``
value of RED or GREEN with equal probability. Finally, Dishonest 
wasps of low aggression get a signal of RED or YELLOW with equal 
probability.
    '''
    
    def __init__(self,params,ID,aggression,character):
        self.LIAR = 0
        self.HONEST = 1
        self.HIGHAG = 2
        self.MEDAG = 1
        self.LOWAG = 0
        self.RED = 2
        self.YELLOW = 1
        self.GREEN = 0
        self.ID = ID
        self.aggression = aggression  ## Can be 0 or 1 or 2
        self.character = character ## Can be 0 or 1, liar or honest
        self.testfield = 0
        self.metabolicCostFactor = params.metCostFactors[self.aggression]
        self.birthday = 0
        self.actuarials = []
        self.characterHistory = [character]
        # Now we set the signal of the agent, based upon cast and honesty.
        if self.character == self.HONEST: # you are honest        
            if self.aggression == self.HIGHAG:
                signal = self.RED
            elif self.aggression == self.MEDAG:
                signal = self.YELLOW
            else:
                signal = self.GREEN
        if self.character == self.LIAR: # you are dishonest
            if self.aggression == self.HIGHAG:
                if random.random() < 0.5:
                    signal = self.YELLOW
                else:
                    signal = self.GREEN
            elif self.aggression == self.MEDAG:
                if random.random() < 0.5:
                    signal = self.RED
                else:
                    signal = self.GREEN
            else:
                if random.random() < 0.5:
                    signal = self.YELLOW
                else:
                    signal = self.RED 
                    
        self.signal = signal
        self.resources = params.aggressionResources[self.aggression]
        
    def getAggression(self):
        if self.aggression == 2:
            return "HIGH"
        elif self.aggression == 1:
            return "MEDIUM"
        elif self.aggression == 0:
            return "LOW"
        else:
            return "AFU"
    def getSignal(self):
        if self.signal == 2:
            return "RED"
        elif self.signal == 1:
            return "YELLOW"
        elif self.signal == 0:
            return "GREEN"
        else:
            return "AFU"
            
    def payMetabolicCosts(self,params):
        self.resources = self.resources - (self.resources * 
            params.metCostFactors[self.aggression])

In [4]:
def create_wasps_waspcastes(params):
    '''
    params is an instance of the Parameters class. Assume it is 
    defined first.
    '''
    # This code is duplicated in sweep().
    if not os.path.isdir(params.outputRoot):
        os.mkdir(params.outputRoot)
    if not os.path.isdir(params.outputFolder):
        os.mkdir(params.outputFolder)
    # reset the tick counter, which applies to individual runs.
    params.tick = 0
    # reset the wasps dictionary
    wasps = {}
    ######### Reset the castes dictionary #######
    waspcastes = {} # keys: level of aggression, values: lists of agents
    # waspcastes of record the IDs of the wasps by level of aggression.
    waspcastes[0] = []
    waspcastes[1] = []
    waspcastes[2] = []
    ###########
    for i in range(params.popSize):
        if i % 3 == 0:
            aggression = 2
        elif i % 2 == 0:
            aggression = 1
        else:
            aggression = 0
        if random.random() < 0.85:
            character = 0
        else:
            character = 1
    
        awasp = wasp(params,i,aggression,character)
        wasps[i] = awasp     
        waspcastes[aggression].append(i)
    f=open(params.outputFolder + os.sep + 'runDataLong'+ \
           str(params.runCount) + 
              '-' + str(params.replicationCount)+'.csv','w')

    headers = 'Tick,PopulationSize,TotalLiars,TotalHonest,TotalOthers,'
    headers += 'Count_0_wasps,Caste_0_Liars,Caste_0_Honest,'
    headers += 'Caste_0_LowAg,Caste_0_MedAg,Caste_0_HiAg,'
    headers += 'signal_0_0,signal_0_0_mean_res,'
    headers += 'signal_0_1,signal_0_1_mean_res,'
    headers += 'signal_0_2,signal_0_2_mean_res,'
    headers += 'Count_1_wasps,Caste_1_Liars,Caste_1_Honest,'
    headers += 'Caste_1_LowAg,Caste_1_MedAg,Caste_1_HiAg,'
    headers += 'signal_1_0,signal_1_0_mean_res,'
    headers += 'signal_1_1,signal_1_1_mean_res,'
    headers += 'signal_1_2,signal_1_2_mean_res,'
    headers += 'Count_2_wasps,Caste_2_Liars,Caste_2_Honest,'
    headers += 'Caste_2_LowAg,Caste_2_MedAg,Caste_2_HiAg,'
    headers += 'signal_2_0,signal_2_0_mean_res,'
    headers += 'signal_2_1,signal_2_1_mean_res,'
    headers += 'signal_2_2,signal_2_2_mean_res,'
    headers += 'time_at_completion'
    f.write(headers + '\n')
    f.close()

    return (wasps,waspcastes)

In [5]:
def recordPopulationStatistics(params,wasps,waspcastes):
    f=open(params.outputFolder + os.sep + 'runDataLong' + 
           str(params.runCount) + 
           '-'  + str(params.replicationCount) +'.csv','a')
    f.write(getPopulationStatistics(params,wasps,waspcastes).strip(',')) # + '\n')

    f.write(',' + str(datetime.datetime.now()) + '\n')
    
    f.close()

In [6]:
def getPopulationStatistics(params,wasps,waspcastes):
    '''
    Returns a list of statistics for the current population of
    wasps. See getCharacterCount().
    
    Assumes params, instance of Parameters, exists.
    '''

    # To begin, get overall data for the total population:
    liarCount = 0
    honestCount = 0
    othersCount = 0
    for i in wasps.keys():
        if wasps[i].character == wasps[i].LIAR:
            liarCount += 1
        elif wasps[i].character == wasps[i].HONEST:
            honestCount += 1
        else:
            othersCount += 1
    dataList = []
    dataList.append(params.tick)
    populationSize = len(wasps)
    dataList.append(populationSize)
    totalLiars = liarCount
    dataList.append(totalLiars)
    totalHonest = honestCount
    dataList.append(totalHonest)
    totalOthers = othersCount
    dataList.append(totalOthers)
    # Next, get data for castes 0,, 1, 2 that is, the low, med, 
    # hi aggression guys.
    ################################  
    for casteID in list(waspcastes.keys()):
        # lists of resources by signaling type:
        redResources = []
        yellowResources = []
        greenResources = []
        liarCount = 0
        honestCount = 0
        for i in waspcastes[casteID]:
            if wasps[i].character == 0:    
                liarCount += 1 
            else:
                honestCount += 1
        dataList.append(len(waspcastes[casteID]))
        dataList.append(liarCount)
        dataList.append(honestCount)
        ####################
        lowagCount = 0
        mediumagCount = 0
        hiagCount = 0
        for i in waspcastes[casteID]:
            if wasps[i].aggression == 0:
                lowagCount += 1
            if wasps[i].aggression == 1:
                mediumagCount += 1
            if wasps[i].aggression == 2:
                hiagCount += 1
        dataList.append(lowagCount)
        dataList.append(mediumagCount)
        dataList.append(hiagCount)
        ####################
        greenCount=0
        yellowCount=0
        redCount=0
        for i in waspcastes[casteID]:
            if wasps[i].signal == 0:
                greenCount += 1
                greenResources.append(wasps[i].resources)
            if wasps[i].signal == 1:
                yellowCount += 1
                yellowResources.append(wasps[i].resources)
            if wasps[i].signal == 2:
                redCount += 1
                redResources.append(wasps[i].resources)
        dataList.append(greenCount)
        if len(greenResources) > 0:
            dataList.append(sum(greenResources)/len(greenResources))
        else:
            dataList.append(0)
        dataList.append(yellowCount)
        if len(yellowResources) > 0:
            dataList.append(sum(yellowResources)/len(yellowResources))
        else:
            dataList.append(0)
        dataList.append(redCount)
        if len(redResources) > 0:
            dataList.append(sum(redResources)/len(redResources))
        else:
            dataList.append(0)
    # Now convert the list of numbers to a CSV string:
    toReturn = makeCSVString(dataList)
    return toReturn

def makeCSVString(listOfNumbers):
    toReturn = ''
    for item in listOfNumbers:
        toReturn += str(item)+','
    return toReturn

### Initialize 
    
Create ``params``, a ``Parameters()`` instance, ``wasps``, a dictionary of ``wasp`` instances,
and ``waspcastes`` a dictionary with keys: wasp castes, values: wasp IDs for wasps in the castes.

In [7]:
def initialize():
    '''
    Initialize the parameters.
    '''
    return Parameters()

In [8]:
params = initialize()

In [9]:
vars(params)

{'aggressionResList': [1.0, 0.95, 0.9, 0.85],
 'aggressionResources': {0: 50, 1: 75, 2: 100},
 'bdlogfile': 'birthanddeathlog.csv',
 'bdlogging': False,
 'metCostFactors': {0: 0.05, 1: 0.075, 2: 0.1},
 'metCostFactorsList': [1.0, 0.95, 0.9, 0.85],
 'numReplications': 10,
 'outputFolder': '../outputs/Run2016-09-122017-09-01 13:42:47.620204/RunDataMulti',
 'outputRoot': '../outputs/Run2016-09-122017-09-01 13:42:47.620204',
 'popSize': 3000,
 'recordEvery': 1000,
 'recordRunDiagnostics': True,
 'recordRunParameters': True,
 'replicationCount': 0,
 'resourceDisHonestAppropriations': {0: 0.5, 1: 0.75, 2: 1.0},
 'resourceDisHonestAppropriationsList': [1.0, 0.95, 0.9, 0.85],
 'resourceHonestAppropriations': {0: 0.25, 1: 0.5, 2: 0.75},
 'resourceHonestAppropriationsList': [1.0, 0.95, 0.9, 0.85],
 'runCount': 0,
 'runDiagnosticsFile': 'rundiagnostics.csv',
 'runLengthInTicks': 1000000,
 'runParametersLog': 'runparameterslog.txt',
 'tick': 0}

### Setup


In [11]:
def setup(params):  
    '''
    runNum is the run ID from the series initiated by ourBSpace().
    wasps is a dictionary with
    keys: IDs and values: wasp objects.
    Calls the class wasp to create individual wasp instances.
    '''
    (wasps,waspcastes) = create_wasps_waspcastes(params)

    f=open(params.outputFolder + os.sep + 'runDataLong'+str(params.runCount) + 
    '-' + str(params.replicationCount)+'.csv','w')

    headers = 'Tick,PopulationSize,TotalLiars,TotalHonest,TotalOthers,'
    headers += 'Count_0_wasps,Caste_0_Liars,Caste_0_Honest,'
    headers += 'Caste_0_LowAg,Caste_0_MedAg,Caste_0_HiAg,'
    headers += 'signal_0_0,signal_0_0_mean_res,'
    headers += 'signal_0_1,signal_0_1_mean_res,'
    headers += 'signal_0_2,signal_0_2_mean_res,'
    headers += 'Count_1_wasps,Caste_1_Liars,Caste_1_Honest,'
    headers += 'Caste_1_LowAg,Caste_1_MedAg,Caste_1_HiAg,'
    headers += 'signal_1_0,signal_1_0_mean_res,'
    headers += 'signal_1_1,signal_1_1_mean_res,'
    headers += 'signal_1_2,signal_1_2_mean_res,'
    headers += 'Count_2_wasps,Caste_2_Liars,Caste_2_Honest,'
    headers += 'Caste_2_LowAg,Caste_2_MedAg,Caste_2_HiAg,'
    headers += 'signal_2_0,signal_2_0_mean_res,'
    headers += 'signal_2_1,signal_2_1_mean_res,'
    headers += 'signal_2_2,signal_2_2_mean_res,'
    #headers += 'aggressionRes,'
    #headers += 'metCostFactors,'
    #headers += 'resourceHonestAppropriations,'
    #headers += 'resourceDisHonestAppropriations,'
    headers += 'time_at_completion'
    f.write(headers + '\n')
    f.close()
    recordPopulationStatistics(params,wasps,waspcastes)
    return (wasps,waspcastes)

### Run


In [12]:
def run(params,wasps,waspcastes): 
    '''
    Performs one run of the model, with a fixed set
    of parmeter values.
    
    Formerly: go(numTicks=1,recordevery=1000,runNum=-1, 
    aggressionRes=1.0,metCostFactors=1.0, 
    resourceHonestAppropriations=1.0, 
    resourceDisHonestAppropriations=1.0)
    '''
    params.tick = 0
    for i in range(params.runLengthInTicks):
        interact(params,wasps,waspcastes)
        params.tick += 1
        if params.tick % params.recordEvery == 0:
            recordPopulationStatistics(params,wasps,waspcastes)
    if params.recordRunDiagnostics:
        collectDiagnostics(params,wasps,waspcastes)


In [13]:
def interact(params,wasps,waspcastes):  
    gangstaID = random.randint(0,params.popSize-1) # The guy who will change.
    gangsta = wasps[gangstaID]
    suckaID = random.randint(0,params.popSize-1) # The guy who will not change.
    sucka = wasps[suckaID]    
    fORF = fightOrFlight(params,gangsta,sucka)
    if fORF == "FLIGHT":
        gangsta.payMetabolicCosts(params)
    else:  # "FIGHT"
        #print("fighting")
        BO = battleOutcome(gangsta,sucka)
        truthOrConsequences(params,BO, gangsta, sucka)
    gangstaUpdate(params,wasps,waspcastes,gangsta)

In [14]:
def fightOrFlight(params,thewasp,thecounterparty):
    assert(thewasp.getAggression() in ["HIGH", "MEDIUM", "LOW"])
    if thewasp.getAggression() == "HIGH":
        if thecounterparty.getSignal() == "RED":
            if random.random() < \
            (thewasp.resources/params.aggressionResources[thewasp.aggression]):
                return "FIGHT"
            else:
                return "FLIGHT"
        else:
            return "FIGHT"
    elif thewasp.getAggression() == "MEDIUM":
        if thecounterparty.getSignal() == "RED":
            return "FLIGHT"
        elif thecounterparty.getSignal() == "YELLOW":
            if random.random() < \
            (thewasp.resources/params.aggressionResources[thewasp.aggression]):
                return "FIGHT"
            else:
                return "FLIGHT"
        elif thecounterparty.getSignal() == "GREEN":  
            return "FIGHT"     
    elif thewasp.getAggression() == "LOW":
        if thecounterparty.getSignal() != "GREEN":
            return "FLIGHT"
        else:
            if random.random() < \
            (thewasp.resources/params.aggressionResources[thewasp.aggression]):
                return "FIGHT"
            else:
                return "FLIGHT"



In [15]:
def battleOutcome(thewasp,thecounterparty):  
    assert(thewasp.getAggression() in ["HIGH", "MEDIUM", "LOW"])
    assert(thecounterparty.getAggression() in ["HIGH", "MEDIUM", "LOW"])
    if thewasp.getAggression() == "HIGH":
        if thecounterparty.getAggression() != "HIGH":
            return "VICTORY"
        else:
            if thewasp.resources > thecounterparty.resources:
                return "VICTORY"
            elif thewasp.resources < thecounterparty.resources:
                return "DEFEAT"
            else:
                if random.random() < 0.5:
                    return "VICTORY"
                else:
                    return "DEFEAT"
    elif thewasp.getAggression() == "MEDIUM":
        if thecounterparty.getAggression() == "HIGH":
            return "DEFEAT"
        elif thecounterparty.getAggression() == "MEDIUM":
            if thewasp.resources > thecounterparty.resources:
                return "VICTORY"
            elif thewasp.resources < thecounterparty.resources:
                return "DEFEAT"
            else:
                if random.random() < 0.5:
                    return "VICTORY"
                else:
                    return "DEFEAT" 
        else:
            return "VICTORY"
    else:
        if thecounterparty.getAggression() != "LOW":
            return "DEFEAT"
        else:
            if thewasp.resources > thecounterparty.resources:
                return "VICTORY"
            elif thewasp.resources < thecounterparty.resources:
                return "DEFEAT"
            else:
                if random.random() < 0.5:
                    return "VICTORY"
                else:
                    return "DEFEAT" 

In [16]:
def truthOrConsequences(params,battleOutcome, theWasp, theCounterparty):
    assert(battleOutcome in ["VICTORY", "DEFEAT"])
    if battleOutcome == "VICTORY":
        if theCounterparty.character == theCounterparty.HONEST:
            theWasp.resources += \
            params.resourceHonestAppropriations[theWasp.aggression] * \
                theCounterparty.resources
            
        elif theCounterparty.character == theCounterparty.LIAR:
            theWasp.resources += \
            params.resourceDisHonestAppropriations[theWasp.aggression] * \
                theCounterparty.resources
                
    if battleOutcome == "DEFEAT":
        if theWasp.character == theWasp.HONEST:
            theWasp.resources -= \
            params.resourceHonestAppropriations[theCounterparty.aggression] * \
                theWasp.resources
        elif theWasp.character == theWasp.LIAR:    
            theWasp.resources -= \
            params.resourceDisHonestAppropriations[theCounterparty.aggression] * \
                theWasp.resources

In [17]:
def gangstaUpdate(params,wasps,waspcastes,theGuy):
    # Are you dead? 
    if theGuy.resources < \
    params.metCostFactors[theGuy.aggression]* \
    params.aggressionResources[theGuy.aggression]:
        yourCrowd = waspcastes[theGuy.aggression]
        withoutYou = yourCrowd[:yourCrowd.index(theGuy.ID)] + \
        yourCrowd[yourCrowd.index(theGuy.ID)+1:]
        
        remaining = len(withoutYou)-1
        leftyIndex = random.randint(0,remaining)
        leftyID = withoutYou[leftyIndex]
        lefty = wasps[leftyID]
        
        rightyIndex = random.randint(0,remaining)
        rightyID = withoutYou[rightyIndex]
        righty = wasps[rightyID]
        if lefty.resources > righty.resources:
            winner = lefty
        else:
            winner = righty
            
        # log the guy
        if params.bdlogging:
            toLog = str(theGuy.ID)+','+str(theGuy.aggression)+','
            toLog += str(theGuy.character)+','+str(theGuy.signal)
            toLog += ','+str(theGuy.birthday)+','+str(params.tick)
        
        theGuy.character = winner.character
        ch = theGuy.characterHistory
        ch.append(theGuy.character)
        theGuy.characterHistory = ch
        theGuy.signal = winner.signal
        theGuy.resources = params.aggressionResources[theGuy.aggression]
        lifespan = params.tick-theGuy.birthday
        act = theGuy.actuarials
        act.append(lifespan)
        theGuy.actuarials = act
        theGuy.birthday = params.tick
        if params.bdlogging:
            toLog += ','+str(theGuy.ID)+','+str(theGuy.aggression)+','
            toLog += str(theGuy.character)+','+str(theGuy.signal)+','+str(theGuy.resources)
            toLog += ','+str(lifespan)+','+str(len(theGuy.actuarials))+','+str(theGuy.characterHistory)     
            logfile = open(params.outputRoot+ os.sep + params.bdlogfile,'a')
            logfile.write(toLog+'\n')
            logfile.close()

vars(params)

In [18]:
def replicate(params,wasps,waspcastes):
    '''
    '''
    for arun in range(params.numReplications):
        # Record the replication count in the parameters object.
        params.replicationCount = arun
        # Create the wasps and waspcastes.
        (wasps,waspcastes) = setup(params)
        '''
        Formerly:
        go(numTicks,runNum=run,aggressionRes=aggressionRes,
           metCostFactors=metCostFactors,
           resourceHonestAppropriations=resourceHonestAppropriations,
           resourceDisHonestAppropriations=resourceDisHonestAppropriations)
        '''
        run(params,wasps,waspcastes)
        

In [19]:
def scaleDict(daDict,daFactor):
    '''
    Alters the parameter by multiplying it by the scale factor for it.
    Returns a new dictionary.
    '''
    #global params
    daNewDict = {}
    for daKey in daDict.keys():
        daNewDict[daKey] = daDict[daKey] * daFactor
    return daNewDict

In [20]:
def collectDiagnostics(params,wasps,waspcastes):
    diagnosticsfile = open(params.outputRoot+ os.sep + params.runDiagnosticsFile,'a')
    for waspCaste in waspcastes.keys():
        # theWasps are a list of the wasp IDs in caste waspCaste
        theWasps = waspcastes[waspCaste]
        # liar-truther: count of wasps who began by lying and 
        # ended as truth tellers, etc.
        liarLiar = 0
        liarTruther = 0
        trutherLiar = 0
        trutherTruther = 0
        liarLiarResources = []
        liarTrutherResources = []
        trutherLiarResources = []
        trutherTrutherResources = []
        liarLiarAlive = []
        liarTrutherAlive = []
        trutherLiarAlive = []
        trutherTrutherAlive = []
        # Now run thru the wasps and collect data
        for wasp in theWasps:
            if wasps[wasp].characterHistory[0] == 0 and wasps[wasp].characterHistory[-1] == 0:
                liarLiar += 1
                liarLiarResources.append(wasps[wasp].resources)
                liarLiarAlive.append(params.tick - wasps[wasp].birthday)
            if wasps[wasp].characterHistory[0] == 0 and wasps[wasp].characterHistory[-1] == 1:
                liarTruther += 1
                liarTrutherResources.append(wasps[wasp].resources)
                liarTrutherAlive.append(params.tick - wasps[wasp].birthday)
            if wasps[wasp].characterHistory[0] == 1 and wasps[wasp].characterHistory[-1] == 0:
                trutherLiar += 1
                trutherLiarResources.append(wasps[wasp].resources)
                trutherLiarAlive.append(params.tick - wasps[wasp].birthday)
            if wasps[wasp].characterHistory[0] == 1 and wasps[wasp].characterHistory[-1] == 1:
                trutherTruther += 1
                trutherTrutherResources.append(wasps[wasp].resources)
                trutherTrutherAlive.append(params.tick - wasps[wasp].birthday)
        strToWrite = str(waspCaste)+','+str(liarLiar)+','+str(liarTruther)+','
        strToWrite += str(trutherLiar)+','+str(trutherTruther)+','
        strToWrite += str(protectedmean(liarLiarResources))+','+str(protectedmean(liarTrutherResources))+','
        strToWrite += str(protectedmean(trutherLiarResources))+','+ str(protectedmean(trutherTrutherResources))+','
        strToWrite += str(protectedmean(liarLiarAlive))+','+str(protectedmean(liarTrutherAlive))+','
        strToWrite += str(protectedmean(trutherLiarAlive))+','+ str(protectedmean(trutherTrutherAlive))+','
        #print(strToWrite)
        diagnosticsfile.write(strToWrite)
    diagnosticsfile.write('\n')
    diagnosticsfile.flush()
    diagnosticsfile.close()
    
def protectedmean(x):
    if x == []:
        return 0
    else:
        return np.mean(x)

In [21]:
def sweep(paramsSource):
    '''
    Replaces its analog, ourBS() in the previous code.
    '''
    
    print(paramsSource.runDiagnosticsFile)
    print(paramsSource.outputRoot)
    if not os.path.isdir(paramsSource.outputRoot):
        os.mkdir(paramsSource.outputRoot)
    #outputFolder = outputRoot + os.sep + 'RunDataMulti' 
    if not os.path.isdir(paramsSource.outputFolder):
        os.mkdir(paramsSource.outputFolder)
    if paramsSource.recordRunDiagnostics:
        print(paramsSource.outputRoot+ os.sep + paramsSource.runDiagnosticsFile)
        diagnosticsfile = open(paramsSource.outputRoot+ os.sep + paramsSource.runDiagnosticsFile,'a')
        listToWrite = ['waspCaste','liarLiar','liarTruther','trutherLiar','trutherTruther',
        'protectedmean(liarLiarResources))','protectedmean(liarTrutherResources))',
        'protectedmean(trutherLiarResources))','protectedmean(trutherTrutherResources))',
        'protectedmean(liarLiarAlive))','protectedmean(liarTrutherAlive))',
        'protectedmean(trutherLiarAlive))','protectedmean(trutherTrutherAlive))']
        # Here is where we print the column headers to the file:
        strToWrite = ''
        for i in paramsSource.aggressionResources.keys():
            for j in listToWrite:
                strToWrite += str(i)+j +','
        diagnosticsfile.write(strToWrite+'\n')
        diagnosticsfile.flush()
        diagnosticsfile.close()

    if paramsSource.bdlogging:
        # Write the headers for the birth and death log file.
        # paramsSource.pdlogfile='birthanddeathlog.csv'
        logfile = open(paramsSource.outputRoot+ os.sep + paramsSource.bdlogfile,'w')
        logfileheader = 'theGuy.ID'+','+'theGuy.aggression'+','+'theGuy.character'+','+'theGuy.signal'
        logfileheader += ','+'theGuy.birthday'+','+'tick'
        logfileheader +=  ','+'theGuy.ID'+','+'theGuy.aggression'+','+'theGuy.character'+','+'theGuy.signal'+','+'theGuy.resources'
        logfileheader += ','+'lifespan'  + ',' + 'rebirths'+','+'characterHistory'
        logfile.write(logfileheader+'\n')   
        logfile.flush()
    # Make a copy of the initialized parameters dictionary.
    params = copy.deepcopy(paramsSource) 
    for i0 in paramsSource.aggressionResList:
        # Scale the dictionary by i0.
        params.aggressionResources = scaleDict(paramsSource.aggressionResources,i0)
        #print(params.aggressionResources)
        # Scale the dictionary by i1.
        for i1 in paramsSource.metCostFactorsList:
            params.metCostFactors = scaleDict(paramsSource.metCostFactors,i1)
            #print(params.metCostFactors)
            for i2 in paramsSource.resourceHonestAppropriationsList:
                params.resourceHonestAppropriations = \
                scaleDict(paramsSource.resourceHonestAppropriations,i2)
                #print(params.resourceHonestAppropriations)
                for i3 in paramsSource.resourceDisHonestAppropriationsList:
                    params.resourceDisHonestAppropriations = \
                    scaleDict(paramsSource.resourceDisHonestAppropriations,i3)
                    
                    print('****** new run %d, %2.2f, %2.2f, %2.2f, %2.2f.' % 
                       (params.runCount,i0,i1,i2,i3))
                    (wasps,waspcastes)=setup(params)
                    replicate(params,wasps,waspcastes)
                    collectDiagnostics(params,wasps,waspcastes)
                    if paramsSource.recordRunParameters:
                        openStr = paramsSource.outputRoot + os.sep 
                        openStr += paramsSource.runParametersLog
                        f = open(openStr,'a')
                        f.write(str(vars(params)))
                        f.flush()
                        f.close()
                    params.runCount += 1
    



In [ ]:
# This is the main procedure. Do the runs by executing go().
# Normally use sweep instead of the run + replicate combination
def go():
    # to initialize:
    theParameters = initialize()
    #####################################
    # Optionally modify the (default) values in theParameters
    theParameters.aggressionResList = [0.4,0.3,0.1]
    theParameters.metCostFactorsList = [0.4,0.3,0.1]
    theParameters.resourceDisHonestAppropriationsList = [0.4,0.3,0.1]
    theParameters.resourceHonestAppropriationsList = [0.4,0.3,0.1]
    theParameters.numReplications = 30
    theParameters.runLengthInTicks = 2000000
    print(vars(theParameters))
    #####################################
    # to setup: 
    (wasps,waspcastes)=setup(theParameters)
    # to run: run(theParameters,wasps,waspcastes)
    # to replicate:     
    # for testing: random.seed(123); random.randint(1,1000)
    # replicate(theParameters,wasps,waspcastes)
    # to sweep:         
    sweep(theParameters)

# Sandbox

go()
